In [5]:
# Imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import shap

from sklearn.ensemble import IsolationForest
from ucimlrepo import fetch_ucirepo

# Set figure background to white
plt.rcParams.update({'figure.facecolor':'white'})

# Data Cleaning and Feature Engineering

In [ ]:
# Fetch dataset from UCI repository
power_consumption = fetch_ucirepo(id=235)

print(power_consumption.variables) 

In [ ]:
# Get all features
data = power_consumption.data.features
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

# List of features to check
feature_columns = ['Global_active_power', 'Global_reactive_power', 'Voltage', 
                   'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']

# Convert feature columns to numeric and replace any errors with NaN
data[feature_columns] = data[feature_columns].apply(pd.to_numeric, errors='coerce')

# Drop rows where all feature columns are missing (NaN) 
data_cleaned = data.dropna(subset=feature_columns, how='all')

# Drop rows where ALL feature columns are NaN
data_cleaned.head()

In [ ]:
# Group by 'Date' and calculate mean and standard deviation (ignore NaN values)
data_aggregated = data_cleaned.groupby('Date')[feature_columns].agg(['mean', 'std'])

# Rename columns to the desired format (MEAN_ColumnName, STD_ColumnName)
data_aggregated.columns = [
    f'{agg_type.upper()}_{col}' for col, agg_type in data_aggregated.columns
]

# Reset the index
data_aggregated.reset_index(inplace=True)

# Display the result
print(data_aggregated.shape)
data_aggregated.head()

# Train IsolationForest

In [10]:
# Parameters
n_estimators = 100  # Number of trees
sample_size = 256  # Number of samples used to train each tree
contamination = 0.02  # Expected proportion of anomalies

In [ ]:
# Select Features
features = data_aggregated.drop('Date', axis=1)

# Train Isolation Forest
iso_forest = IsolationForest(n_estimators=n_estimators, 
                             contamination=contamination, 
                             max_samples=sample_size,
                             random_state=42)

iso_forest.fit(features)

In [ ]:
data_aggregated['anomaly_score'] = iso_forest.decision_function(features)
data_aggregated['anomaly'] = iso_forest.predict(features)

data_aggregated['anomaly'].value_counts()

In [ ]:
# Visualization of the results
plt.figure(figsize=(10, 5))

# Plot normal instances
normal = data_aggregated[data_aggregated['anomaly'] == 1]
plt.scatter(normal['Date'], normal['anomaly_score'], label='Normal')

# Plot anomalies
anomalies = data_aggregated[data_aggregated['anomaly'] == -1]
plt.scatter(anomalies['Date'], anomalies['anomaly_score'], label='Anomaly')

plt.xlabel("Instance")
plt.ylabel("Anomaly Score")
plt.legend()

# KernelSHAP with Anomaly Score


In [ ]:
# Using the anomaly score and TreeSHAP (this code won't work)
explainer = shap.TreeExplainer(iso_forest.decision_function, features)
shap_values = explainer(features)

In [ ]:
# Select all anomalies and 100 random normal instances
normal_sample = np.random.choice(normal.index,size=100,replace=False)
sample = np.append(anomalies.index,normal_sample)

len(sample) # 129

In [ ]:
# Using the anomaly score and KernelSHAP
explainer = shap.Explainer(iso_forest.decision_function, features)
shap_values = explainer(features.iloc[sample])

In [ ]:
# Plot waterfall plot of an anomaly
shap.plots.waterfall(shap_values[0])

In [ ]:
# Plot waterfall plot of a normal instance
shap.plots.waterfall(shap_values[100])

In [ ]:
# MeanSHAP Plot
shap.plots.bar(shap_values)

In [ ]:
# Beeswarm plot
shap.plots.beeswarm(shap_values)

# TreeSHAP with Path Length

In [22]:
# Calculate SHAP values
explainer = shap.TreeExplainer(iso_forest)
shap_values = explainer(features)

In [ ]:
# Waterfall plot for an anomaly
shap.plots.waterfall(shap_values[0])

In [ ]:
# Waterfall plot for a normal instance
shap.plots.waterfall(shap_values[2])

In [ ]:
# Calculate f(x)
path_length = shap_values.base_values + shap_values.values.sum(axis=1)

# Get f(x) for anomalies and normal instances
anomalies = data_aggregated[data_aggregated['anomaly'] == -1]
path_length_anomalies = path_length[anomalies.index]

normal = data_aggregated[data_aggregated['anomaly'] == 1]
path_length_normal = path_length[normal.index]

# Plot boxplots for f(x)
plt.figure(figsize=(10, 5))
plt.boxplot([path_length_anomalies, path_length_normal], labels=['Anomaly','Normal'])
plt.ylabel("Average Path Length f(x)")

In [ ]:
# MeanSHAP
shap.plots.bar(shap_values)

In [ ]:
# MeanSHAP
shap.plots.beeswarm(shap_values)

In [26]:
# Interaction values
shap_interaction_values = explainer.shap_interaction_values(features)

In [ ]:
# Get absolute mean of matrices
mean_shap = np.abs(shap_interaction_values).mean(0)
mean_shap = np.round(mean_shap, 1)

df = pd.DataFrame(mean_shap, index=features.columns, columns=features.columns)

# Times off diagonal by 2
df.where(df.values == np.diagonal(df), df.values * 2, inplace=True)

# Display
sns.set(font_scale=1)
sns.heatmap(df, cmap="coolwarm", annot=True)
plt.yticks(rotation=0)